In [78]:
%matplotlib inline

from IPython.display import Audio
from IPython.display import Image

import tensorflow as tf
from tensorflow import keras
import os
import numpy as np 
import librosa as lr
import librosa.display
import wave
import soundfile as sf
import math
import json

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

This notebook reads one of the given validation datasets and makes a training dataset out of it.\
\
Validation samples are divided into 3 second intervals in the following manner. The number of intervals is calculated as:
$$n\_ int=\lceil{duration\div 3}\rceil.$$
First $n\_ int-1$ intervals is taken by dividing the first $3\times (n\_ int-1)$ seconds into $n\_ int-1$ non-overlapping intervals. And the last 3 seconds of the sample are taken as the last interval.

It saves that data into a numpy tensor and its corresponding labels as well.

Enter your username or change your path below. 
Enter the instrument for which you want to save labels as well.\
\
Each data tensor should only be made once per validation dataset. The labels should be saved for each instrument separately.

In [79]:
username = "user"
instrument_of_interest = "gel"

broj = 3

directory = "C:/Users/" + username + "/Downloads/IRMAS_Validation_Data_part" + str(broj) + "/IRMAS_Validation_Data_part" + str(broj)
wav_files = [f for f in os.listdir(directory) if f.endswith('.wav')]
validation_files = [os.path.join(directory, f) for f in wav_files]

m=len(validation_files)

i=0
n_coef = 20
hop_length = 1024

tensor = np.empty((0, n_coef, 130))

# Print the final tensor shape
print(tensor.shape)

n_interval_dict = {}
start_index_dict = {}

for file in validation_files:
    
    audio = wave.open(file)
    duration_seconds = audio.getnframes()/audio.getframerate()
    n_intervals = math.ceil(duration_seconds/3)
    
    n_interval_dict[file] = n_intervals # mozdda da stavis samo n_interval_dict[file], ako su svi fileovi drukcijeg naziva
    
    
    for j in range(n_intervals-1):
        
        y, sr = librosa.load(file, mono=True, sr=None, offset=j*3.0, duration=3.0)

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_coef, hop_length=hop_length)
        
        tensor = np.concatenate((tensor, np.expand_dims(mfcc, axis=0)), axis=0)
        
        if (j==0):
            
            start_index_dict[file] = tensor.shape[0] - 1 # PROVJERITI
    
    y, sr = librosa.load(file, mono=True, sr=None, offset=duration_seconds-3.0, duration=3.0)
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_coef, hop_length=1024)
    
    tensor = np.concatenate((tensor, np.expand_dims(mfcc, axis=0)), axis=0)
   
        
    i = i+1
    print(i)

(0, 20, 130)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [80]:
np.save(str(broj) + 'TENZOR_mfcc_NESKALIRANI_HW=1024_VALI' + str(broj) + '.npy', tensor)

In [66]:
print(m)

989


The next cell creates a .json file which contains the starting index for each interval.

In [67]:
filename = str(broj) + 'index_dict.json'
with open(filename, 'w') as file:
    json.dump(start_index_dict, file)

# Later, to load the dictionary from the file
with open(filename, 'r') as file:
    loaded_start_index_dict = json.load(file)

The next cell creates a .json file which contains the duration of each interval.

In [68]:
filename = str(broj) + 'n_interval_dict.json'
with open(filename, 'w') as file:
    json.dump(n_interval_dict, file)

# Later, to load the dictionary from the file
with open(filename, 'r') as file:
    loaded_n_intervals= json.load(file)

In [70]:
loaded_start_index_dict

{'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-1.wav': 0,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-11.wav': 7,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-12.wav': 11,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-13.wav': 14,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-14.wav': 17,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-15.wav': 22,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-2.wav': 27,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-3.wav': 29,
 'C:/Users/et

In [71]:
loaded_n_intervals

{'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-1.wav': 7,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-11.wav': 4,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-12.wav': 3,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-13.wav': 3,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-14.wav': 5,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-15.wav': 5,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-2.wav': 2,
 'C:/Users/etnubac/Downloads/IRMAS_Validation_Data_part1/IRMAS_Validation_Data_part1\\(02) dont kill the whale-3.wav': 3,
 'C:/Users/etnubac/

In [72]:
# Determine ground truth labels for validation data
ground_truth_labels = []
for file in validation_files:
    with open(os.path.splitext(file)[0] + '.txt', 'r') as f:
        text = f.read()
    if instrument_of_interest in text:
        ground_truth_labels.append(1)
    else:
        ground_truth_labels.append(0)
ground_truth_labels = np.array(ground_truth_labels)

print(ground_truth_labels)
print(ground_truth_labels.shape)

[1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1
 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

The next cell shows how the labeling of the validation data works on a simple example.\
\
We *assumed* that if our instrument of interest is labeled as existant in a 20.0 second audio file for example, we will label it as existant in each of its 7 intervals. That is, we will create 7 training samples with label 1. Procedure is analogous when the original label is 0, as shown below.

In [54]:
label_array = [0, 1, 0]
dictionary = {"a": 3, "b": 5, "c": 4}

final_output = []
index = 0  # Initialize index
for label in label_array:
    key = list(dictionary.keys())[index]
    value = dictionary[key]
    final_output.extend([label] * value)
    index += 1  # Increment index

print(final_output)


[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]


Below we create final labels for the given tensor and instrument of interest.

In [73]:
final_output = []
index = 0  # Initialize index
for label in ground_truth_labels:
    key = list(loaded_n_intervals.keys())[index]
    value = loaded_n_intervals[key]
    final_output.extend([label] * value)
    index += 1  # Increment index

print(final_output)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

We will save the labels as an numpy array for future use. The name indicates the instrument of interest and the number of the validation dataset.

In [75]:
np.save(instrument_of_interest + '_' + str(broj) + 'labels_mfcc_NESKALIRANI_HW=1024_VALI' + str(broj) + '.npy', final_output)

In [76]:
unique, counts = np.unique(final_output, return_counts = True)
print(unique)
print(counts)

[0 1]
[4170 1943]
